# Scale Structure based on C-C distances

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.data.structure import StructureData
from aiida.orm.data.base import Float
from aiida.work.workfunction import workfunction

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
from structure_browser import StructureBrowser

## Step 1: Select a structure from the AiiDA database

In [ ]:
def on_struct_change(c):
    global atoms, scaling_factor
    node = struct_browser.results.value
    if node:
        atoms = node.get_ase()
        scaling_factor.value = guess_scaling_factor(atoms) if atoms else None
    else:
        atoms = None
        scaling_factor.value = 0.0

def on_factor_change(c):
    if struct_browser.results.value:
        inp_descr.value = struct_browser.results.value.description + ".scale(%.3f)"%scaling_factor.value
    else:
        inp_descr.value = ""

struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
scaling_factor = ipw.FloatText(description="scaling factor", step=0.001)
scaling_factor.observe(on_factor_change, names='value')

clear_output()
display(struct_browser, scaling_factor)

In [ ]:
def guess_scaling_factor(atoms):
    # set bounding box as cell
    cx = 1.5 * (np.amax(atoms.positions[:,0]) - np.amin(atoms.positions[:,0]))
    cy = 1.5 * (np.amax(atoms.positions[:,1]) - np.amin(atoms.positions[:,1]))
    cz = 15.0
    atoms.cell = (cx, cy, cz)
    atoms.pbc = (True,True,True)
    
    # calculate all atom-atom distances
    c_atoms = [a for a in atoms if a.symbol[0]=="C"]
    n = len(c_atoms)
    dists = np.zeros([n,n])
    for i, a in enumerate(c_atoms):
        for j, b in enumerate(c_atoms):
            dists[i,j] = norm(a.position - b.position)
            
    # find bond distances to closest neighbor
    dists += np.diag([np.inf]*n) # don't consider diagonal
    bonds = np.amin(dists, axis=1)
    
    # average bond distance
    avg_bond = float(scipy.stats.mode(bonds)[0])
    
    # scale box to match equilibrium carbon-carbon bond distance
    cc_eq = 1.4313333333
    s = cc_eq / avg_bond
    return s

## Step 2: Store structure in the AiiDA database

In [ ]:
def on_click_store(b):
    global atoms, removals
    
    orig_struct = struct_browser.results.value
    s = scale(orig_struct, Float(scaling_factor.value))
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))
    
inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))

In [ ]:
# using a workfunction to create link to original structure
@workfunction
def scale(orig_struct, scaling_factor):
    atoms = orig_struct.get_ase()
    cx, cy, cz = atoms.cell
    s = scaling_factor.value
    atoms.set_cell((s*cx, s*cy, cz), scale_atoms=True)
    atoms.center()
    return StructureData(ase=atoms)